In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

# SST 5

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
from collections import defaultdict


model_id = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

# SST-5 template from paper
SST5_TEMPLATE = "Text: {text}\nSentiment: {label}\n\n"

def format_example(example):
    return SST5_TEMPLATE.format(
        text=example["text"],
        label=example["label"]
    )

def get_label_probs(prompt, text):
    full_input = prompt + f"Text: {text}\nSentiment:"
    inputs = tokenizer(full_input, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs).logits

    # Label tokens (adjust based on tokenizer's mapping)
    label_tokens = [tokenizer(f" {i}")["input_ids"][-1] for i in range(5)]
    last_token_logits = outputs[0, -1, label_tokens]
    return torch.softmax(last_token_logits, dim=-1).cpu().numpy()

def evaluate_subset(Si, val_set):
    prompt = "".join([format_example(ex) for ex in Si])
    correct = 0
    for ex in val_set:
        probs = get_label_probs(prompt, ex["text"])
        if np.argmax(probs) == ex["label"]:
            correct += 1
    return correct / len(val_set)

def compute_influences(train_set, val_set, k=5, M=100):
    # Balanced subset sampling
    class_counts = defaultdict(int)
    for ex in train_set:
        class_counts[ex["label"]] += 1

    k_per_class = max(1, k // len(class_counts))
    subsets = []
    for _ in range(M):
        Si = []
        for label in class_counts:
            candidates = [ex for ex in train_set if ex["label"] == label]
            Si.extend(np.random.choice(candidates, k_per_class, replace=False))
        Si = list(np.random.choice(Si, k, replace=False))
        subsets.append(Si)

    # Sequential evaluation
    D = []
    for Si in tqdm(subsets, desc="Evaluating subsets"):
        D.append(evaluate_subset(Si, val_set))

    # Calculate influences
    influence_scores = defaultdict(list)
    for idx, ex in enumerate(train_set):
        included = []
        excluded = []
        for Si, acc in zip(subsets, D):
            if ex in Si:
                included.append(acc)
            else:
                excluded.append(acc)

        Nj = len(included)
        Mj = len(D) - Nj
        if Nj > 0 and Mj > 0:
            influence = (sum(included)/Nj) - (sum(excluded)/Mj)
        else:
            influence = 0
        influence_scores[idx] = influence

    return influence_scores

def load_sst5(split="train", num_samples=1000):
    dataset = load_dataset("SetFit/sst5", "default", split=split)
    dataset = dataset.shuffle(seed=42).select(range(min(num_samples, len(dataset))))
    return dataset

def run_experiment():
    # Load SST-5
    train_set = load_sst5("train", 5000)
    val_set = load_sst5("validation", 1000)
    test_set = load_sst5("test", 1000)

    # Compute influences
    influence_scores = compute_influences(train_set, val_set, k=5, M=100)

    # Select top examples
    sorted_indices = sorted(influence_scores, key=influence_scores.get, reverse=True)[:5]
    top_examples = [train_set[i] for i in sorted_indices]

    # Final evaluation
    prompt = "".join([format_example(ex) for ex in top_examples])
    correct = 0
    for ex in tqdm(test_set, desc="Testing"):
        probs = get_label_probs(prompt, ex["text"])
        if np.argmax(probs) == ex["label"]:
            correct += 1

    print(f"Final Test Accuracy: {correct/len(test_set):.2%}")

if __name__ == "__main__":
    run_experiment()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Testing: 100%|██████████| 1000/1000 [00:21<00:00, 47.17it/s]

Final Test Accuracy: 28.70%


# AG News

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

def load_agnews():
    splits = {'train': 'data/train-00000-of-00001.parquet',
             'test': 'data/test-00000-of-00001.parquet'}

    # Load and sample data
    df_train_full = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["train"])
    df_sampled = df_train_full.groupby('label', group_keys=False).apply(
        lambda x: x.sample(n=1500, random_state=42)).reset_index(drop=True)

    # Train/val split
    df_train, df_val = train_test_split(
        df_sampled,
        test_size=1000,
        stratify=df_sampled['label'],
        random_state=42
    )

    # Test set
    df_test_full = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["test"])
    df_test = df_test_full.sample(n=1000, random_state=42)

    # Convert to HuggingFace datasets
    train_set = Dataset.from_pandas(df_train[['text', 'label']])
    val_set = Dataset.from_pandas(df_val[['text', 'label']])
    test_set = Dataset.from_pandas(df_test[['text', 'label']])

    return train_set, val_set, test_set
def load_agnews():
    splits = {'train': 'data/train-00000-of-00001.parquet',
             'test': 'data/test-00000-of-00001.parquet'}

    # Load and sample data
    df_train_full = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["train"])
    df_sampled = df_train_full.groupby('label', group_keys=False).apply(
        lambda x: x.sample(n=1500, random_state=42)).reset_index(drop=True)

    # Train/val split
    df_train, df_val = train_test_split(
        df_sampled,
        test_size=1000,
        stratify=df_sampled['label'],
        random_state=42
    )

    # Test set
    df_test_full = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["test"])
    df_test = df_test_full.sample(n=1000, random_state=42)

    # Convert to HuggingFace datasets
    train_set = Dataset.from_pandas(df_train[['text', 'label']])
    val_set = Dataset.from_pandas(df_val[['text', 'label']])
    test_set = Dataset.from_pandas(df_test[['text', 'label']])

    return train_set, val_set, test_set

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import pandas as pd


model_id = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

AGNEWS_TEMPLATE = "Text: {text}\nClass label: {label}\n\n"

def format_example(example):
    return AGNEWS_TEMPLATE.format(
        text=example["text"],
        label=example["label"]
    )

def get_label_probs(prompt, text):
    full_input = prompt + f"Text: {text}\nSentiment:"
    inputs = tokenizer(full_input, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs).logits

    # Label tokens (adjust based on tokenizer's mapping)
    label_tokens = [tokenizer(f" {i}")["input_ids"][-1] for i in range(5)]
    last_token_logits = outputs[0, -1, label_tokens]
    return torch.softmax(last_token_logits, dim=-1).cpu().numpy()

def evaluate_subset(Si, val_set):
    prompt = "".join([format_example(ex) for ex in Si])
    correct = 0
    for ex in val_set:
        probs = get_label_probs(prompt, ex["text"])
        if np.argmax(probs) == ex["label"]:
            correct += 1
    return correct / len(val_set)

def compute_influences(train_set, val_set, k=5, M=100):
    # Balanced subset sampling
    class_counts = defaultdict(int)
    for ex in train_set:
        class_counts[ex["label"]] += 1

    k_per_class = max(1, k // len(class_counts))
    subsets = []
    for _ in range(M):
        Si = []
        for label in class_counts:
            candidates = [ex for ex in train_set if ex["label"] == label]
            Si.extend(np.random.choice(candidates, k_per_class, replace=False))
        Si = list(np.random.choice(Si, k, replace=True))
        subsets.append(Si)

    # Sequential evaluation
    D = []
    for Si in tqdm(subsets, desc="Evaluating subsets"):
        D.append(evaluate_subset(Si, val_set))

    # Calculate influences
    influence_scores = defaultdict(list)
    for idx, ex in enumerate(train_set):
        included = []
        excluded = []
        for Si, acc in zip(subsets, D):
            if ex in Si:
                included.append(acc)
            else:
                excluded.append(acc)

        Nj = len(included)
        Mj = len(D) - Nj
        if Nj > 0 and Mj > 0:
            influence = (sum(included)/Nj) - (sum(excluded)/Mj)
        else:
            influence = 0
        influence_scores[idx] = influence

    return influence_scores

def run_experiment():
    train_set, val_set, test_set = load_agnews()

    # Compute influences
    influence_scores = compute_influences(train_set, val_set, k=5, M=100)

    # Select top examples
    sorted_indices = sorted(influence_scores, key=influence_scores.get, reverse=True)[:5]
    top_examples = [train_set[i] for i in sorted_indices]

    # Final evaluation
    prompt = "".join([format_example(ex) for ex in top_examples])
    correct = 0
    for ex in tqdm(test_set, desc="Testing"):
        probs = get_label_probs(prompt, ex["text"])
        if np.argmax(probs) == ex["label"]:
            correct += 1

    print(f"Final Test Accuracy: {correct/len(test_set):.2%}")

if __name__ == "__main__":
    run_experiment()


<ipython-input-9-c408396fe45c>:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_train_full.groupby('label', group_keys=False).apply(
Testing: 100%|██████████| 1000/1000 [00:35<00:00, 27.99it/s]

Final Test Accuracy: 30.60%
